# Helsinki Bike Data
---
This notebook is used to clean the Helsinki bike data.

In [115]:
"""
Comment out the following lines if you have already installed the packages.
"""
# !pip install pandas
# !pip install tqdm

'\nComment out the following lines if you have already installed the packages.\n'

## Section : Define functions
---
- get_data
- drop_columns
- replace_chars_on_column
- change_column_data_type
- change_column_names
- drop_distances_and_durations_under_10
- concat_dataframes
- change_time_format
- dataframe_to_csv
- sort_by_column
- combine_lat_long
- merge_station_data_and_bike_data

### Read the function docstrings for more in depth information about the functions.

In [116]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path


def get_data(data_path: str):
    """
    Reads the csv file at the given path and returns a dataframe.

    Parameters:
    data_path (str): path to the csv file

    Returns:
    pd.DataFrame: dataframe with the data from the csv file
    """
    return pd.read_csv(data_path)


def drop_columns(dataframe: pd.DataFrame, columns_to_drop: list):
    """
    Drops the given columns from the input dataframe.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be modified
    columns_to_drop (list): list of columns to be dropped

    Returns:
    pd.DataFrame: processed dataframe with the dropped columns
    """
    dataframe.drop(columns=columns_to_drop, inplace=True)
    return dataframe


def replace_chars_on_column(dataframe: pd.DataFrame):
    """
    Replaces the characters in the column names with underscores and lowercases the column names.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be modified

    Returns:
    pd.DataFrame: processed dataframe with the renamed columns
    """
    string_list_to_replace_with_blank = [".", "(", ")"]
    dataframe.columns = dataframe.columns.str.replace(' ', '_', regex=True).str.lower()
    for char in string_list_to_replace_with_blank:
        dataframe.columns = dataframe.columns.str.replace(char, '', regex=True)

    return dataframe


def change_column_data_type(dataframe: pd.DataFrame):
    """
    Change the data type of the columns 'covered_distance_m' and 'duration_sec' to int.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be modified

    Returns:
    pd.DataFrame: dataframe with the changed data types
    """
    dataframe.covered_distance_m = dataframe.covered_distance_m.astype(int)
    dataframe.duration_sec = dataframe.duration_sec.astype(int)
    return dataframe


def change_column_names(dataframe, columns_to_change: dict):
    """
    Changes the column names in the input dataframe to the given column names.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be modified
    columns_to_change (dict): dictionary with the old column names as keys and the new column names as values

    Returns:
    pd.DataFrame: dataframe with the changed column names
    """
    dataframe.rename(columns=columns_to_change, inplace=True)
    return dataframe


def drop_distances_and_durations_under_10(dataframe: pd.DataFrame):
    """ Drops rows where covered_distance_m is less than 10 and duration_sec is less than 10.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be cleaned

    Returns:
    pd.DataFrame: cleaned dataframe

    """
    dataframe_size = len(dataframe)

    dataframe.drop_duplicates(inplace=True)
    dataframe = dataframe[(dataframe.covered_distance_m >= 10) & (dataframe.duration_sec >= 10)].copy()
    dataframe = dataframe[dataframe.covered_distance_m.notna()].copy()

    dataframe_new_size = len(dataframe)

    print(f"Number of rows dropped: {dataframe_size - dataframe_new_size}\n{'-' * 50}")

    return dataframe


def concat_dataframes(array_of_dataframes: list):
    """ Concatenates the input dataframes into one dataframe.

    Parameters:
    array_of_dataframes (list): list of dataframes to be concatenated

    Returns:
    pd.DataFrame: concatenated dataframe

    """
    return pd.concat(array_of_dataframes).reset_index(drop=True).reset_index(drop=True)


def change_time_format(dataframe: pd.DataFrame):
    """ Changes the format of the 'start_time' and 'end_time' columns in the input dataframe.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be modified

    Returns:
    pd.DataFrame: dataframe with the changed time format
    """
    time = ['start_time', 'end_time']
    for when in time:
        dataframe[when] = pd.to_datetime(dataframe[when], format='%Y-%m-%d %H:%M:%S')

    return dataframe


def dataframe_to_csv(dataframe: pd.DataFrame, filename: str, indx: bool) -> None:
    """ Saves the input dataframe to a csv file with the given filename.
    When indx is True, the index column will be saved to the csv file.
    Also convert 'start_time' and 'end_time' columns in dataframe to datetime.
    We use this since SQL Lite wants an index column but Postgres does not.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be saved
    filename (str): name of the file
    indx (bool): True if index is to be saved, False if not

    Returns:
    None
    """
    if indx:
        dataframe.index.name = "id"
        dataframe.index += 1

    if filename == "helsinki_bike_data":
        dataframe = change_time_format(dataframe)

    print(f"Saving dataframe, please wait...\n")
    dataframe.to_csv(Path.cwd() / 'data' / f"{filename}_new.csv", index=indx, sep=";", encoding='utf-8', header=True)

    try:
        print(f"Dataframe saved to {filename}_new.csv")
    except FileNotFoundError:
        print("File not found")


def sort_by_column(dataframe: pd.DataFrame, sort_by: str):
    """ Sorts the input dataframe by the given column name.

    Parameters:
    dataframe (pd.DataFrame): dataframe to be sorted
    sort_by (str): column name to sort by

    Returns:
    pd.DataFrame: sorted dataframe
    """
    return dataframe.sort_values(by=[sort_by], ascending=True).reset_index(drop=True)


def combine_lat_long(row, start: bool = True):
    if start:
        return f"{row['start_station_latitude']}, {row['start_station_longitude']}"
    else:
        return f"{row['end_station_latitude']}, {row['end_station_longitude']}"


def merge_station_data_and_bike_data(bike_data: pd.DataFrame, station_data: pd.DataFrame):
    """ Merges the input dataframes into one dataframe.

    Merged columns:
    id, start_time, end_time, start_station_id, start_station_name, start_station_location, end_station_id, end_station_name, end_station_location, distance_m, duration_s

    Parameters:
    bike_data (pd.DataFrame): dataframe containing bike data
    station_data (pd.DataFrame): dataframe containing station data

    Returns:
    pd.DataFrame: merged dataframe
    """
    tqdm.pandas()
    merged_df = pd.merge(bike_data, station_data, left_on="start_station_id", right_on="station_id", how="left")
    merged_df = pd.merge(merged_df, station_data, left_on="end_station_id", right_on="station_id", how="left")
    merged_df = merged_df.rename(columns={"name_x": "start_station_name", "address_x": "start_station_address",
                                          "longitude_x": "start_station_longitude",
                                          "latitude_x": "start_station_latitude",
                                          "name_y": "end_station_name", "address_y": "end_station_address",
                                          "longitude_y": "end_station_longitude", "latitude_y": "end_station_latitude"})
    merged_df = merged_df.drop(
        columns=["station_id_x", "station_id_y", "end_station_address", "start_station_address"])

    print(
        f"\nCombining latitude and longitude into one column, start_station_location and end_station_location\n{'-' * 100}")
    merged_df["start_station_location"] = merged_df.progress_apply(lambda row: combine_lat_long(row, start=True),
                                                                   axis=1)
    merged_df["end_station_location"] = merged_df.progress_apply(lambda row: combine_lat_long(row, start=False), axis=1)

    merged_df = merged_df.drop(
        columns=["start_station_latitude", "start_station_longitude", "end_station_latitude", "end_station_longitude"])

    merged_df = merged_df.reindex(
        columns=["start_time", "end_time", "start_station_id", "start_station_name", "start_station_location",
                 "end_station_id", "end_station_name", "end_station_location",
                 "distance_m", "duration_s"])

    return merged_df


# Section: Inspect the original data

In [117]:
get_data('./data/2021-05.csv').head(10)

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2021-05-31T23:57:25,2021-06-01T00:05:46,94,Laajalahden aukio,100,Teljäntie,2043.0,500
1,2021-05-31T23:56:59,2021-06-01T00:07:14,82,Töölöntulli,113,Pasilan asema,1870.0,611
2,2021-05-31T23:56:44,2021-06-01T00:03:26,123,Näkinsilta,121,Vilhonvuorenkatu,1025.0,399
3,2021-05-31T23:56:23,2021-06-01T00:29:58,4,Viiskulma,65,Hernesaarenranta,4318.0,2009
4,2021-05-31T23:56:11,2021-06-01T00:02:02,4,Viiskulma,65,Hernesaarenranta,1400.0,350
5,2021-05-31T23:54:48,2021-06-01T00:00:57,292,Koskelan varikko,133,Paavalinpuisto,1713.0,366
6,2021-05-31T23:54:11,2021-06-01T00:17:11,34,Kansallismuseo,81,Stenbäckinkatu,2550.0,1377
7,2021-05-31T23:53:04,2021-06-01T00:14:52,240,Viikin normaalikoulu,281,Puotila (M),5366.0,1304
8,2021-05-31T23:52:03,2021-06-01T00:15:16,116,Linnanmäki,117,Brahen puistikko,3344.0,1393
9,2021-05-31T23:50:19,2021-06-01T00:05:58,116,Linnanmäki,145,Pohjolankatu,3248.0,935


In [118]:
get_data('./data/2021-06.csv').head(10)

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2021-06-30T23:59:46,2021-06-30T23:59:55,41,Ympyrätalo,41,Ympyrätalo,0.0,8
1,2021-06-30T23:59:36,2021-07-01T00:06:21,107,Tenholantie,111,Esterinportti,1847.0,407
2,2021-06-30T23:59:34,2021-07-01T00:06:23,9,Erottajan aukio,40,Hakaniemi (M),1602.0,405
3,2021-06-30T23:59:34,2021-07-01T00:08:00,113,Pasilan asema,107,Tenholantie,2277.0,502
4,2021-06-30T23:59:25,2021-07-01T00:22:15,41,Ympyrätalo,11,Unioninkatu,4010.0,1366
5,2021-06-30T23:59:25,2021-07-01T00:14:06,259,Petter Wetterin tie,248,Gunillantie,3555.0,877
6,2021-06-30T23:59:21,2021-07-01T00:24:48,391,Tollinpolku,387,Kauppakeskus Kaari,4337.0,1526
7,2021-06-30T23:59:07,2021-07-01T00:16:00,46,Diakoniapuisto,67,Perämiehenkatu,4265.0,1008
8,2021-06-30T23:59:02,2021-07-01T00:04:05,280,Puotilan ostoskeskus,278,Puotilantie,855.0,302
9,2021-06-30T23:58:54,2021-07-01T00:22:04,72,Eteläinen Hesperiankatu,121,Vilhonvuorenkatu,4627.0,1385


In [119]:
get_data('./data/2021-07.csv').head(10)

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2021-07-31T23:59:59,2021-08-01T00:09:15,113,Pasilan asema,78,Messeniuksenkatu,1602.0,553
1,2021-07-31T23:59:55,2021-08-01T00:08:45,135,Velodrominrinne,115,Venttiilikuja,1307.0,532
2,2021-07-31T23:59:55,2021-08-01T00:03:24,258,Abraham Wetterin tie,260,Herttoniemi (M),820.0,205
3,2021-07-31T23:59:47,2021-08-01T00:05:52,122,Lintulahdenkatu,16,Liisanpuistikko,1298.0,369
4,2021-07-31T23:59:33,2021-08-01T00:14:49,126,Kalasatama (M),255,Laivalahden puistotie,3875.0,912
5,2021-07-31T23:59:28,2021-08-01T00:06:23,115,Venttiilikuja,43,Karhupuisto,1482.0,410
6,2021-07-31T23:59:22,2021-08-01T00:06:14,347,Sateenkaarentie,344,Puistolan asema,1698.0,410
7,2021-07-31T23:59:11,2021-08-01T00:06:11,347,Sateenkaarentie,344,Puistolan asema,1658.0,416
8,2021-07-31T23:59:01,2021-08-01T00:09:56,21,Töölönlahdenkatu,58,Lauttasaarensilta,3150.0,649
9,2021-07-31T23:58:35,2021-08-01T00:23:48,87,Kustaankatu,95,Munkkiniemen aukio,4835.0,1509


In [120]:
get_data('./data/station_data.csv').head(10)

,FID,ID,Nimi,Namn,Name,Osoite,Adress,Kaupunki,Stad,Operaattor,Kapasiteet,x,y
0,1,501,Hanasaari,Hanaholmen,Hanasaari,Hanasaarenranta 1,Hanaholmsstranden 1,Espoo,Esbo,CityBike Finland,10,24.840319,60.165820
1,2,503,Keilalahti,Kägelviken,Keilalahti,Keilalahdentie 2,Kägelviksvägen 2,Espoo,Esbo,CityBike Finland,28,24.827467,60.171524
2,3,505,Westendinasema,Westendstationen,Westendinasema,Westendintie 1,Westendvägen 1,Espoo,Esbo,CityBike Finland,16,24.805758,60.168266
3,4,507,Golfpolku,Golfstigen,Golfpolku,Golfpolku 3,Golfstigen 3,Espoo,Esbo,CityBike Finland,16,24.796136,60.168143
4,5,509,Revontulentie,Norrskensvägen,Revontulentie,Revontulentie 10,Norrskensvägen 10,Espoo,Esbo,CityBike Finland,30,24.802938,60.171551
5,6,511,Sateentie,Regnvägen,Sateentie,Sateentie 2,Regnvägen 2,Espoo,Esbo,CityBike Finland,18,24.810688,60.173424
6,7,513,Hakalehto,Hagliden,Hakalehto,Merituulentie 18,Havsvindsvägen 18,Espoo,Esbo,CityBike Finland,24,24.791390,60.173567
7,8,515,Oravannahkatori,Gråskinnstorget,Oravannahkatori,Oravannahkatori 1,Gråskinnstorget 1,Espoo,Esbo,CityBike Finland,16,24.792559,60.175769
8,9,517,Länsituuli,Västanvinden,Länsituuli,Länsituulenkuja 3,Västanvindsgränden 3,Espoo,Esbo,CityBike Finland,24,24.802049,60.175358
9,10,518,Tuulimäki,Väderbacken,Tuulimäki,Itätuulenkuja 11,Östanvindsgränden 11,Espoo,Esbo,CityBike Finland,18,24.806051,60.174144


# Section: Run the functions
Process bikedata (2021-05, 2021-06, 2021-07) and save the dataframes to csv files.

In [121]:
"""
Section: Run the functions
1. Process bikedata (2021-05, 2021-06, 2021-07) and save the dataframes to csv files.
"""

print(f"Cleaning bikedata\n{'-' * 100}")
dataframe_1 = change_column_data_type(drop_columns(
    dataframe=drop_distances_and_durations_under_10(replace_chars_on_column(get_data('./data/2021-05.csv'))),
    columns_to_drop=['departure_station_name', 'return_station_name']))
dataframe_2 = drop_columns(
    dataframe=drop_distances_and_durations_under_10(replace_chars_on_column(get_data('./data/2021-06.csv'))),
    columns_to_drop=['departure_station_name', 'return_station_name'])
dataframe_3 = drop_columns(
    dataframe=drop_distances_and_durations_under_10(replace_chars_on_column(get_data('./data/2021-07.csv'))),
    columns_to_drop=['departure_station_name', 'return_station_name'])

merged_dataframe = concat_dataframes([dataframe_1, dataframe_2, dataframe_3])

merged_dataframe = change_column_names(merged_dataframe,
                                       columns_to_change={'covered_distance_m': 'distance_m',
                                                          'duration_sec': 'duration_s',
                                                          "departure_station_id": "start_station_id",
                                                          "return_station_id": "end_station_id",
                                                          "departure": "start_time",
                                                          "return": "end_time"})

"""
2. Process station_data.csv and save the dataframe to a csv file.
"""
stations = drop_columns(
    replace_chars_on_column(dataframe=get_data('./data/station_data.csv')),
    columns_to_drop=["nimi", "fid", "stad", "operaattor", "namn", "kapasiteet", "adress", "kaupunki"])
station_data = change_column_names(
    stations, {"id": "station_id", "osoite": "address", "y": "latitude", "x": "longitude"}
)

merged_dataframe = merge_station_data_and_bike_data(bike_data=merged_dataframe, station_data=stations)

merged_dataframe = sort_by_column(merged_dataframe, 'start_time')
dataframe_to_csv(dataframe=merged_dataframe, filename="helsinki_bike_data", indx=True)


Cleaning bikedata
----------------------------------------------------------------------------------------------------
Number of rows dropped: 422027
--------------------------------------------------
Number of rows dropped: 634062
--------------------------------------------------
Number of rows dropped: 626534
--------------------------------------------------

Combining latitude and longitude into one column, start_station_location and end_station_location
----------------------------------------------------------------------------------------------------


100%|██████████| 1564379/1564379 [00:12<00:00, 125350.10it/s]


Saving dataframe, please wait...

Dataframe saved to helsinki_bike_data_new.csv


## Section: Inspect random rows from the modified dataframes

In [122]:
dataframe_1.sample(5)

,departure,return,departure_station_id,return_station_id,covered_distance_m,duration_sec
130123,2021-05-22T21:30:05,2021-05-22T21:39:26,5,64,1764,557
5502,2021-05-31T17:01:57,2021-05-31T17:13:22,118,124,2197,679
382659,2021-05-03T10:23:44,2021-05-03T10:32:03,43,122,635,494
403189,2021-05-01T12:46:19,2021-05-01T13:27:32,653,539,5905,2469
150339,2021-05-21T17:14:54,2021-05-21T17:36:28,64,55,4887,1293


In [123]:
dataframe_2.sample(5)

,departure,return,departure_station_id,return_station_id,covered_distance_m,duration_sec
544623,2021-06-04T12:51:49,2021-06-04T13:12:18,349,339,4039.0,1224
368970,2021-06-12T12:42:05,2021-06-12T13:08:12,214,113,4692.0,1563
235427,2021-06-18T22:19:57,2021-06-18T22:31:13,591,581,2068.0,672
232056,2021-06-19T02:25:01,2021-06-19T02:45:18,111,103,2991.0,1211
30284,2021-06-29T16:21:30,2021-06-29T16:35:43,103,208,3320.0,852


In [124]:
dataframe_3.sample(5)

,departure,return,departure_station_id,return_station_id,covered_distance_m,duration_sec
194944,2021-07-20T16:42:16,2021-07-20T16:45:50,705,705,753.0,213
266965,2021-07-16T17:43:03,2021-07-16T17:47:49,161,8,508.0,282
600718,2021-07-01T09:52:30,2021-07-01T10:01:22,39,87,1868.0,528
223975,2021-07-18T21:07:34,2021-07-18T21:23:39,32,128,3910.0,961
544321,2021-07-03T17:20:29,2021-07-03T17:26:24,707,727,1341.0,351


In [125]:
station_data.sample(5)

,station_id,name,address,longitude,latitude
286,225,Maunula,Maunulanmäki 2,24.932503,60.230383
5,511,Sateentie,Sateentie 2,24.810688,60.173424
246,139,Kaironkatu,Arabiankatu 21,24.979704,60.209710
0,501,Hanasaari,Hanasaarenranta 1,24.840319,60.165820
374,325,Mellunmäki (M),Pallaksentie 3,25.109875,60.238535


In [126]:
merged_dataframe.sample(10)

,start_time,end_time,start_station_id,start_station_name,start_station_location,end_station_id,end_station_name,end_station_location,distance_m,duration_s
id,,,,,,,,,,
184353,2021-05-16 12:41:00,2021-05-16 13:00:02,144,Käpyläntie,"60.2139042918024, 24.9588108000086",231,Siltavoudintie,"60.2337172192606, 24.9649692401006",3361.0,1136
898690,2021-06-26 02:07:55,2021-06-26 02:34:22,3,Kapteeninpuistikko,"60.1581769029949, 24.9450181631667",26,Kamppi (M),"60.168609492141, 24.9305373005364",1586.0,467
478094,2021-06-05 12:17:36,2021-06-05 12:22:11,613,Matinkyläntie,"60.155689, 24.744978",615,Tiistiläntie,"60.156755, 24.737664",656.0,272
1537103,2021-07-29 21:01:57,2021-07-29 21:09:46,123,Näkinsilta,"60.179750209022, 24.9554449266327",124,Isoisänsilta,"60.1826518123339, 24.9817926053222",1843.0,468
1130342,2021-07-07 20:55:10,2021-07-07 21:04:43,208,Valimotie,"60.2159693222985, 24.8765290693421",106,Korppaanmäentie,"60.2034734922503, 24.898929699376",2150.0,568
452961,2021-06-04 07:43:05,2021-06-04 07:54:44,61,Länsisatamankatu,"60.158927591706, 24.9096920006591",19,Central Railway Station/East,"60.1708235917114, 24.9425267992462",2987.0,698
1373911,2021-07-20 14:04:57,2021-07-20 14:26:57,60,Cable Factory,"60.161252848, 24.90279681",501,Hanasaari,"60.16582, 24.840319",4079.0,1318
889056,2021-06-25 13:27:44,2021-06-25 14:39:31,311,Ramsinniementie,"60.1995636503578, 25.1199060794359",301,Aurinkotuulenkatu,"60.202773395376, 25.1593463483962",6328.0,4301
790780,2021-06-20 15:19:28,2021-06-20 15:27:13,91,Seurasaari,"60.1884345423222, 24.8847643214706",96,Huopalahdentie,"60.199732919393, 24.8835764865762",1388.0,462
